<a href="https://colab.research.google.com/github/NaveenSanjaya/Deepfake-Face-Detection-In-The-Wild/blob/main/GAN_Deepfake_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms

## Step 3: Dataset Preparation

In [2]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load dataset (use your dataset path)
dataset = datasets.ImageFolder('/content/drive/MyDrive/Projects/IEEE - SP Cup/data', transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)


## Step 4: Define the Feature Extractor

In [3]:
# Load pretrained ResNet
feature_extractor = models.resnet50(pretrained=True)
feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-1])  # Remove last layer
feature_extractor.eval()  # Freeze parameters

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 109MB/s]


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


## Step 5: Define the Discriminator

In [4]:
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

discriminator = Discriminator(input_dim=2048)  # ResNet50 feature size


## Step 6: Define Loss and Optimizer

In [5]:
criterion = nn.BCELoss()  # Binary cross-entropy loss
optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)


## Step 7: Training the Framework

In [6]:
# Training loop
for epoch in range(5):
    for images, labels in dataloader:
        # Extract features
        features = feature_extractor(images).view(images.size(0), -1)

        # Create real and fake labels
        real_labels = torch.ones(features.size(0), 1).to(features.device)
        fake_labels = torch.zeros(features.size(0), 1).to(features.device)

        # Train discriminator
        real_outputs = discriminator(features)
        loss_real = criterion(real_outputs, real_labels)

        # Assume fake_features are generated somehow (for now, use random noise)
        fake_features = torch.randn_like(features)
        fake_outputs = discriminator(fake_features)
        loss_fake = criterion(fake_outputs, fake_labels)

        # Backpropagation
        loss = loss_real + loss_fake
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{10}, Loss: {loss.item()}")


Epoch 1/10, Loss: 1.0487792491912842
Epoch 2/10, Loss: 0.8780028223991394
Epoch 3/10, Loss: 0.7988165020942688
Epoch 4/10, Loss: 0.6804363131523132
Epoch 5/10, Loss: 0.6765204668045044


## Step 8: Save the Model

In [7]:
torch.save(discriminator.state_dict(), "discriminator.pth")

## Step 9: Testing the Model

In [10]:
# Load a test image
from PIL import Image
import torchvision.transforms.functional as F

test_image = Image.open('/content/drive/MyDrive/Projects/IEEE - SP Cup/data/fake/valid_fake_0000523.png')  # Example image
test_image = F.resize(test_image, (224, 224))
test_image = F.to_tensor(test_image).unsqueeze(0)

# Extract features
with torch.no_grad():
    test_features = feature_extractor(test_image).view(1, -1)
    prediction = discriminator(test_features)

# Output prediction
# Interpret prediction
threshold = 0.5  # Classify as real if probability > 0.5
if prediction.item() > threshold:
    result = "Real"
    confidence = prediction.item() * 100  # Convert to percentage
else:
    result = "Fake"
    confidence = (1 - prediction.item()) * 100  # Convert to percentage

# Display result
print(f"Prediction: {result} (Confidence: {confidence:.2f}%)")



Prediction: Real (Confidence: 97.07%)
